In [37]:
import pandas as pd
import numpy as np

In [10]:
raw_data=pd.read_csv("C:\machihe learning\Projects\AI shoping agent\dataset\Online_Retail.csv", encoding='latin-1')

In [11]:
# --- STEP 2: THE CLEANING FUNCTION ---
def clean_retail_data(df):
    print("Starting data cleaning...")
    
    # Drop missing CustomerIDs
    df = df.dropna(subset=['CustomerID'])
    
    # Remove cancellations and non-positive values
    df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
    df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
    
    # Format types
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    df['CustomerID'] = df['CustomerID'].astype(int)
    
    # Create TotalSum
    df['TotalSum'] = df['Quantity'] * df['UnitPrice']
    
    # Remove duplicates
    df = df.drop_duplicates()
    
    print(f"Cleaning complete. New dataset shape: {df.shape}")
    return df

In [12]:
# --- STEP 3: RUN AND CHECK ---
df_clean = clean_retail_data(raw_data)

# Check the baseline numbers for Phase 2
print("\n--- BASELINE CHECKS ---")
print(f"Date Range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Unique Customers: {df_clean['CustomerID'].nunique()}")
print(f"Total Revenue: ${df_clean['TotalSum'].sum():,.2f}")

Starting data cleaning...


C:\Users\hp\AppData\Local\Temp\ipykernel_9500\4218397572.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


Cleaning complete. New dataset shape: (392692, 9)

--- BASELINE CHECKS ---
Date Range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Unique Customers: 4338
Total Revenue: $8,887,208.89


# RFM Calculation

In [13]:
import datetime as dt

In [14]:
# 1. Set the snapshot date (one day after the last purchase)
snapshot_date = df_clean['InvoiceDate'].max() + dt.timedelta(days=1)


In [15]:
# 2. Aggregate data by CustomerID
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, # Recency
    'InvoiceNo': 'count',                                   # Frequency
    'TotalSum': 'sum'                                       # Monetary
})

In [16]:
# 3. Rename columns for clarity
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'Monetary'
}, inplace=True)

print("RFM Table Created Successfully!")
print(rfm.head())

RFM Table Created Successfully!
            Recency  Frequency  Monetary
CustomerID                              
12346           326          1  77183.60
12347             2        182   4310.00
12348            75         31   1797.24
12349            19         73   1757.55
12350           310         17    334.40


In [17]:
rfm.describe()

,Recency,Frequency,Monetary
count,4338.000000,4338.000000,4338.000000
mean,92.536422,90.523744,2048.688081
std,100.014169,225.506968,8985.230220
min,1.000000,1.000000,3.750000
25%,18.000000,17.000000,306.482500
50%,51.000000,41.000000,668.570000
75%,142.000000,98.000000,1660.597500
max,374.000000,7676.000000,280206.020000


In [18]:
import pandas as pd
import datetime as dt

# Load data (Ensure the filename matches yours)
df = pd.read_csv('C:\machihe learning\Projects\AI shoping agent\dataset\Online_Retail.csv', encoding="ISO-8859-1")

# 1. Professional Cleaning
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalSum'] = df['Quantity'] * df['UnitPrice']

# 2. RFM Calculation
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique', # Count unique orders, not just rows
    'TotalSum': 'sum'
}).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalSum': 'Monetary'})

print("Consultant Foundation Ready!")
print(f"Total Clients Analyzed: {len(rfm)}")


C:\Users\hp\AppData\Local\Temp\ipykernel_9500\153116572.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


Consultant Foundation Ready!
Total Clients Analyzed: 4338


In [19]:
# 1. Define "High Value" (Top 25% of spenders) and "At Risk" (90+ days away)
monetary_threshold = rfm['Monetary'].quantile(0.75) 
recency_threshold = 90


In [20]:
# 2. Filter the "At-Risk Whales"
at_risk_whales = rfm[(rfm['Monetary'] >= monetary_threshold) & (rfm['Recency'] > recency_threshold)]

In [21]:
# 3. Sort them by Monetary value to show the biggest "Leaks" first
at_risk_whales = at_risk_whales.sort_values(by='Monetary', ascending=False)

print("--- REVENUE LEAK REPORT ---")
print(f"Number of High-Value Customers at risk: {len(at_risk_whales)}")
print(f"Total Revenue currently at risk: ${at_risk_whales['Monetary'].sum():,.2f}")
print("\nTop 5 Customers to target for win-back:")
print(at_risk_whales.head())

--- REVENUE LEAK REPORT ---
Number of High-Value Customers at risk: 95
Total Revenue currently at risk: $434,984.93

Top 5 Customers to target for win-back:
            Recency  Frequency  Monetary
CustomerID                              
12346.0         326          1  77183.60
15749.0         235          3  44534.30
15098.0         182          3  39916.50
16180.0         100          8  10254.18
12590.0         211          2   9864.26


In [22]:
# 4. Save to CSV to "hand over" to the client
at_risk_whales.to_csv('High_Value_At_Risk_List.csv')

# Margin Protection

In [23]:
freq_threshold = rfm['Frequency'].quantile(0.75)
recency_low_threshold = rfm['Recency'].quantile(0.25)

In [24]:
champions = rfm[(rfm['Frequency'] >= freq_threshold) & (rfm['Recency'] <= recency_low_threshold)]


In [25]:
print("--- MARGIN PROTECTION REPORT ---")
print(f"Number of 'Champion' Customers: {len(champions)}")
print(f"Total Revenue from Champions: ${champions['Monetary'].sum():,.2f}")
print(f"Average Spend per Champion: ${champions['Monetary'].mean():,.2f}")


--- MARGIN PROTECTION REPORT ---
Number of 'Champion' Customers: 623
Total Revenue from Champions: $4,628,828.07
Average Spend per Champion: $7,429.90


In [26]:
champions.to_csv('Champions_Full_Price_List.csv')

# visualization


In [27]:
import plotly.express as px

In [28]:
# Assign tiers based on the thresholds we used previously
def get_customer_tier(row):
    if row['Monetary'] >= rfm['Monetary'].quantile(0.75) and row['Recency'] <= rfm['Recency'].quantile(0.25):
        return 'Champion (High Value, Low Churn)'
    elif row['Monetary'] >= rfm['Monetary'].quantile(0.75) and row['Recency'] > rfm['Recency'].quantile(0.75):
        return 'At-Risk Whale (Revenue Leak)'
    elif row['Recency'] > rfm['Recency'].quantile(0.75) and row['Frequency'] < rfm['Frequency'].quantile(0.25):
        return 'Hibernating (Low Value/Low Activity)'
    else:
        return 'Average/Potential'

rfm['Tier'] = rfm.apply(get_customer_tier, axis=1)

In [29]:
# Create a Treemap visualization using Plotly (interactive charts)
fig = px.treemap(rfm, 
                 path=['Tier'], 
                 values='Monetary', 
                 color='Recency',
                 color_continuous_scale='RdYlGn_r', # Red-Yellow-Green color scale
                 title='Customer Base Segmentation by Revenue & Recency')
fig.show()

print("Treemap visualization generated successfully!")

Treemap visualization generated successfully!


# Probability of Churn 

In [30]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [31]:
# We'll define churn as not purchasing in the last 90 days
rfm['Churned'] = (rfm['Recency'] > 90).astype(int)

In [32]:
X = rfm[['Recency', 'Frequency', 'Monetary']]
y = rfm['Churned']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# 4. Initialize and Train XGBoost (The 2026 Industry Standard)
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [35]:
# 5. Predict Probabilities for ALL customers
rfm['Churn_Probability'] = model.predict_proba(X)[:, 1]

In [36]:
print("Predictive Model Trained Successfully!")
print(rfm[['Recency', 'Frequency', 'Monetary', 'Churn_Probability']].head())

Predictive Model Trained Successfully!
            Recency  Frequency  Monetary  Churn_Probability
CustomerID                                                 
12346.0         326          1  77183.60           0.998577
12347.0           2          7   4310.00           0.000228
12348.0          75          4   1797.24           0.000228
12349.0          19          1   1757.55           0.000228
12350.0         310          1    334.40           0.999358
